### Import Modules

In [1]:
#1
!pip install -q -U json-lines

import json
import os
import pandas as pd
import tarfile
import json_lines
from tqdm.notebook import tqdm
from pprint import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np

### read party, tweets, mentions and hashtags

In [3]:
#2
#funktioniert: get Party name + Tweets
files123 = glob.glob('./data/*.jl')

partyname_n_tweets = []
partyname_mentions = []
partyname_hash = []
error = []

i = 0

for file in tqdm(files123): # responsible for visual output

    #if(i == 300):
        #continue
    #i += 1
    
    with json_lines.open(file) as f:
        for item in f:
            try:
                if item['response']['meta']['result_count'] == 0: 
                    continue
                elif item['account_data']['Partei'] == "Fraktionslos":
                    continue
                if item['http_status'] == 200:
                    
                    #get party name
                    partei = item["account_data"]["Partei"]
                    
                    #get all tweets per person
                    tweets_array = item["response"]["data"]
                    
                    for tweet in tweets_array: #range(0,len(tweets_array)-1):
                       # print(type(tweet))
                        if("referenced_tweets" in tweet): # tweet is a dictionary
                            tweet_type = tweet["referenced_tweets"]
                            if(tweet_type[0]["type"] == "retweeted"):
                                continue
                        partyname_n_tweets.append({"Partei": partei, "Tweet": tweet["text"]}) # hashtags, mentions in eigene Arrays
                        
                        #if("entities" in tweet): 
                            #entities = tweet["entities"]
                            #if("mentions" in entities):
                                #mentions = entities["mentions"]
                                #for ment in mentions:
                                        #mention = ment["username"]
                                        #partyname_mentions.append({"Partei": partei, "mentions": mention})
                            #if("hashtags" in entities):
                                #hashtags = entities["hashtags"]
                                #for tag in hashtags:
                                        #hashtag = tag["tag"]
                                        #partyname_hash.append({"Partei": partei, "hashtags": hashtag})
                                        
                else:
                    error.append(item)
                    print("yikes")
                    
            except Exception as e:
                raise e
                           
print("Number of error-requests:", len(error))
print("Read", len(partyname_n_tweets), "Tweets sucussfully.")
print("mentions", len(partyname_mentions))
print("hashtags", len(partyname_hash))


Number of error-requests: 0
Read 1806094 Tweets sucussfully.
mentions 0
hashtags 0


In [ ]:
print("Tweets insgesamt", len(partyname_n_tweets))
partyname_n_tweets_df = pd.DataFrame(partyname_n_tweets)
partyname_n_tweets_df["Partei"].value_counts()

In [ ]:
#3
##Versuch auf uneleganter Weise: ----
files123 = glob.glob('./data/*.jl')

partyname_n_tweets_without_retweets = []
error = []

for file in tqdm(files123): # responsible for visual output
    with json_lines.open(file) as f:
        for item in f:
            try:
                if item['response']['meta']['result_count'] == 0: 
                    continue
                elif item['account_data']['Partei'] == "Fraktionslos":
                    continue
                if item['http_status'] == 200:
                    
                    #get party name
                    partei = item["account_data"]["Partei"]
                    
                    #get all tweets per person
                    tweets_array = item["response"]["data"]
                    for tweet in tweets_array:
                      
                        if tweet["text"].startswith("RT @"):
                            continue
                        
                        partyname_n_tweets_without_retweets.append({"Partei": partei, "Tweet": tweet["text"]})
                else:
                    error.append(item)
                    print("yikes")
                    
            except Exception as e:
                raise e
                           
print("Number of error-requests:", len(error))
print("Read", len(partyname_n_tweets_without_retweets), "Tweets sucussfully.")

In [ ]:
#3_1
text_df = pd.DataFrame(partyname_n_tweets_without_retweets)
print(text_df.head())

# Failed H2O

In [ ]:
#3_2
#!apt -get install default-jre
#!java -version
!pip install h2o
#!pip install emojis
!pip install emoji

In [4]:
#3_3
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import emojis
import emoji

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.281-b09, mixed mode)


C:\Users\Runjie\anaconda3\lib\site-packages\h2o\backend\server.py:385: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\Users\Runjie\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Runjie\AppData\Local\Temp\tmp_vkj5c3s
  JVM stdout: C:\Users\Runjie\AppData\Local\Temp\tmp_vkj5c3s\h2o_Runjie_started_from_python.out
  JVM stderr: C:\Users\Runjie\AppData\Local\Temp\tmp_vkj5c3s\h2o_Runjie_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,"28 days, 22 hours and 16 minutes"
H2O_cluster_name:,H2O_from_python_Runjie_71li82
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Runjie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
pnt_df = pd.DataFrame(partyname_n_tweets)
pnt_df = pnt_df.sample(n=141141, random_state = 42)
pnt_df["Partei"].value_counts()

Bündnis 90/Die Grünen    33707
SPD                      27794
CDU                      26134
Die Linke                25681
FDP                      14426
AfD                       7902
CSU                       5497
Name: Partei, dtype: int64

In [9]:
target={"Bündnis 90/Die Grünen":0, "SPD":1, "CDU":2, "Die Linke":3, "FDP":4, "AfD":5, "CSU":6}
pnt_df["target"]=pnt_df["Partei"].map(target)
pnt_df

,Partei,Tweet,target
751143,Bündnis 90/Die Grünen,ist natürlich dabei! Abstimmen für volle Gleic...,0
517497,FDP,@Energisch_ warum nicht! #Luxemburg,4
1552452,Bündnis 90/Die Grünen,Schwarz-Gelb beschließt gegen Opppsition Haush...,0
138650,Bündnis 90/Die Grünen,Sonder-Fraktionssitzung zu #Griechenland,0
750667,Bündnis 90/Die Grünen,"#Grüne NDS: Superergebnisse in Uni-Städten, gr...",0
...,...,...,...
1740624,Bündnis 90/Die Grünen,@WollErnst Sie müssen sich schon sich den Inha...,0
558568,CSU,Wallersdorfer Heimaterde in Berlin. \nEine sch...,6
487369,SPD,Da kann der #Sarrazin bis zum Papst gehen. Ras...,1
700942,CSU,@verenahubertz @fdpbt @EUTheurer Es kann nach ...,6


In [10]:
X_train, X_test = train_test_split(pnt_df, test_size=0.4, random_state=111)

In [11]:
stemmer = nltk.stem.SnowballStemmer("german")
nltk.download("stopwords")
stop_words=set(nltk.corpus.stopwords.words("german"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Runjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
print(stop_words)

{'derselbe', 'würden', 'dazu', 'deiner', 'anderes', 'doch', 'welches', 'dieselbe', 'jene', 'meines', 'viel', 'so', 'ihnen', 'was', 'auch', 'wenn', 'zwar', 'welche', 'wir', 'an', 'anderr', 'andere', 'meinem', 'zum', 'sondern', 'eine', 'waren', 'anders', 'hin', 'sind', 'man', 'deinen', 'werde', 'einen', 'war', 'diesem', 'unter', 'derselben', 'sein', 'bis', 'jeden', 'am', 'manches', 'können', 'seinem', 'damit', 'das', 'oder', 'einigem', 'musste', 'dasselbe', 'ihres', 'des', 'meinen', 'deinem', 'jede', 'haben', 'und', 'ins', 'mich', 'durch', 'eures', 'einig', 'die', 'vor', 'eurer', 'weiter', 'ihre', 'hier', 'bin', 'dann', 'keines', 'seines', 'dein', 'manchen', 'nach', 'zu', 'keinem', 'manche', 'ihr', 'einiger', 'derer', 'in', 'im', 'solches', 'uns', 'wieder', 'jetzt', 'indem', 'soll', 'aber', 'zur', 'allen', 'keinen', 'unseren', 'solchen', 'eurem', 'nur', 'solchem', 'aus', 'dieselben', 'deines', 'welcher', 'anderem', 'daß', 'für', 'hab', 'hat', 'dies', 'habe', 'wird', 'mancher', 'dieser', 

In [13]:
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text)] #if (len(word) > 3 and len(word.strip("Xx/")) > 2 and len(re.sub("\d+", "", word.strip("Xx/")))]
    tokens = map(str.lower, tokens)
    tokens = [word.strip(" #|€!?,.:;'`‘’“”„@[]()=*+-–%/0123456789& ") for word in tokens]
    tokens = [word for word in tokens if not word in emoji.UNICODE_EMOJI]
    ##tokens = [word.strip() for word in tokens]
    stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
    return stems

In [14]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
train_vectors=vectorizer_tf.fit_transform(X_train.Tweet)

In [15]:
train_vectors.A[:20]

array([[0.33343994, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11408344, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.32733697, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.52161588, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31981752, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
vectorizer_tf.get_feature_names()

[' ',
 ' a',
 ' ab',
 ' afd',
 ' afdimbundestag',
 ' akk',
 ' amp',
 ' arbeit',
 ' bay',
 ' beim',
 ' berlin',
 ' bess',
 ' bild',
 ' bildung',
 ' bitt',
 ' bleibt',
 ' brauch',
 ' btw',
 ' bundesregier',
 ' bundestag',
 ' bundeswehr',
 ' c_lindn',
 ' cdu',
 ' cducsubt',
 ' corona',
 ' csu',
 ' d',
 ' dabei',
 ' dafur',
 ' dank',
 ' darf',
 ' demokrati',
 ' deshalb',
 ' deutsch',
 ' deutschland',
 ' die_gru',
 ' dielink',
 ' doroba',
 ' einfach',
 ' endlich',
 ' energiew',
 ' erst',
 ' eu',
 ' euro',
 ' europa',
 ' faznet',
 ' fb',
 ' fdp',
 ' fdpbt',
 ' find',
 ' fluchtling',
 ' frag',
 ' frau',
 ' freiheit',
 ' freu',
 ' ganz',
 ' geht',
 ' gemeinsam',
 ' genau',
 ' gerad',
 ' gibt',
 ' gleich',
 ' groko',
 ' gross',
 ' gruenebundestag',
 ' grun',
 ' gut',
 ' halt',
 ' hatt',
 ' heikomaas',
 ' herzlich',
 ' heut',
 ' http',
 ' https',
 ' imm',
 ' inn',
 ' ja',
 ' jahr',
 ' jensspahn',
 ' kahr',
 ' kind',
 ' klar',
 ' klimaschutz',
 ' komm',
 ' kommt',
 ' konstantinnotz',
 ' land',
 '

In [17]:
test_vectors=vectorizer_tf.transform(X_test.Tweet)

In [18]:
X_train.shape

(84684, 3)

In [19]:
train_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
train_df=pd.concat([train_df,X_train["target"].reset_index(drop=True)], axis=1)
train_df

MemoryError: Unable to allocate 646. MiB for an array with shape (84684, 1000) and data type float64

In [ ]:
test_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
test_df=pd.concat([test_df,X_test["target"].reset_index(drop=True)], axis=1)
test_df

In [ ]:
import win_unicode_console
win_unicode_console.enable()

import sys
import codecs

if sys.stdout.encoding != 'cp850':
  sys.stdout = codecs.getwriter('cp850')(sys.stdout, 'strict')
if sys.stderr.encoding != 'cp850':
  sys.stderr = codecs.getwriter('cp850')(sys.stderr, 'strict')

In [ ]:
h2o_train_df=h2o.H2OFrame(train_df)
h2o_test_df=h2o.H2OFrame(test_df)

In [ ]:
h2o_train_df.types

In [ ]:
h2o_train_df["target"]=h2o_train_df["target"].asfactor()
h2o_test_df["target"]=h2o_test_df["target"].asfactor()

# Data Preprocessing

In [6]:
#imports
import re
import nltk
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

#import ML LIB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

#import Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [7]:
#get stemmer, stop_words
stemmer = nltk.stem.SnowballStemmer("german")
nltk.download("stopwords")
stop_words=set(nltk.corpus.stopwords.words("german"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Runjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#build DataFrame
pnt_df2 = pd.DataFrame(partyname_n_tweets)
pnt_df2

,Partei,Tweet
0,Bündnis 90/Die Grünen,@green_clarity99 Danke dir!
1,Bündnis 90/Die Grünen,Die Industrie ist das Rückgrat unseres Wohlsta...
2,Bündnis 90/Die Grünen,@peter_simone @RenateKuenast Herzlichen Dank l...
3,Bündnis 90/Die Grünen,"Ebenfalls herzlichen Glückwunsch, liebe @Renat..."
4,Bündnis 90/Die Grünen,@BernhardPoetter treffend: „🌍 CO2-Rückgang 202...
...,...,...
1806089,CDU,Mit bis zu 20 Mio € wird das ⁦@BMBF_Bund⁩ künf...
1806090,CDU,Die Simulation ist neben Theorie und Experimen...
1806091,CDU,Schule darf keine „digitalen Analphabeten“ erz...
1806092,CDU,@StefanKaufmann @helmholtz_de 👍


In [23]:
#split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(pnt_df2["Tweet"], pnt_df2.Partei, test_size = 0.3, random_state=111)

In [24]:
X_train

802192     Ja, es dürfen alle kommen #bewegungjetzt und N...
1573011    Volle Zustimmung zu Christoph von Marschall im...
1020826    So, genau zu #MarkusKrebs auf #RTL wieder pünk...
52074      Parlamentarische Mehrheit im Europarat für pal...
639127     Datenabgleich etc. scheitern bestimmt nicht an...
                                 ...                        
102486     @HanseNico @penzium Bin in Wrist, ist vermutli...
135892     @gudruncita @mrohrlack @h_beppo @dadres @Ludwi...
534484     Meine Bundestagsrede zur #Frauenquote als Film...
654188     @LarsWendland @jorg_radek @GdPPresse Nach unse...
1519444    @spektrallinie "In" sagt Volkmar Sigusch. Verm...
Name: cleaned, Length: 1264265, dtype: object

In [25]:
pnt_df2["Partei"].value_counts()

Bündnis 90/Die Grünen    433846
SPD                      354097
CDU                      336243
Die Linke                326787
FDP                      182542
AfD                      102382
CSU                       70197
Name: Partei, dtype: int64

In [26]:
y_train.value_counts()

Bündnis 90/Die Grünen    303776
SPD                      247627
CDU                      235729
Die Linke                228384
FDP                      128032
AfD                       71564
CSU                       49153
Name: Partei, dtype: int64

In [27]:
y_test.value_counts()

Bündnis 90/Die Grünen    130070
SPD                      106470
CDU                      100514
Die Linke                 98403
FDP                       54510
AfD                       30818
CSU                       21044
Name: Partei, dtype: int64

In [28]:
#write our own tokenizer function
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text)] #if (len(word) > 3 and len(word.strip("Xx/")) > 2 and len(re.sub("\d+", "", word.strip("Xx/")))]
    tokens = map(str.lower, tokens)
    tokens = [word.strip(" #|€!?,.:;'`‘’“”„@[]()=*+-–%/0123456789& ") for word in tokens]
    #tokens = [word for word in tokens if not word in emoji.UNICODE_EMOJI]
    #tokens = [word.strip() for word in tokens]
    stems = [item for item in tokens if (item not in stop_words)]
    #stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
    return stems

# Support Vector Classifier

In [29]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", LinearSVC(C=1.0, penalty="l1", max_iter=6000, dual=False))])

In [30]:
model = pipeline.fit(X_train, y_train)

C:\Users\Runjie\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [31]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [32]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices = True)]
feature_names = np.asarray(feature_names)

In [33]:
#target_names = ["Bündnis 90/Die Grünen", "SPD", "CDU", "Die Linke", "FDP", "AfD", "CSU"]
target_names = clf.classes_
print("top 30 keywords per class:")
for i, label in enumerate(target_names):
    top30 = np.argsort(clf.coef_[i])[-30:]
    print("%s: %s" % (label, " ".join(feature_names[top30])))
    print("\n")

top 30 keywords per class:
AfD: merkeldeutschland bessermenschen altparteienstaat repswalwell zwangsgebühren  antifaterrorists jürgen braun grünlinken afdbayern merkelland kestner einheitsmedien  gezabschaffen mrosek intelligenzgibtsnichtimbioladen bernhard https nurnochafd  toterjournalismus ramelüringen afdwählen grünlinke afdimbundestag afd👥 afdimbundestag⁩  afdwirkt  traudichdeutschland klimawahn buntland ardzdf altparteien


Bündnis 90/Die Grünen:  enveryucel  lavonrw rio http  darumgruen erststimmemutlu ms https  selcukrsirin mutlu hocam 🌻🌻🌻 gerhard schick gruenenrw uetd tabeafährtbahn garantierente queer_de zukunftentscheidetsichhier liz_de marc_lueger klimakrise bdk gruenemitte albtour sw http  gruenebundestag greenrunners tt ssr mutludirekt bü


CDU: dülken kreisreform neustaat klimke lindow hdh cducsubt⁩ moltkezitate cdu-kreisverband kommentar woche viersen lb bremen-nord stgt cda neuruppin  euerpaul willicher hohensteinernstthal sachsenring aicgs cducsubt  teamröttgen jetztv

In [35]:
prediction = model.predict(X_test)

In [38]:
print("******SVM classification report (full tweets, all data)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

******SVM classification report (full tweets, all data)******
accuracy: 0.5215501569683424
                       precision    recall  f1-score   support

                  AfD     0.5726    0.6074    0.5895     30818
Bündnis 90/Die Grünen     0.4694    0.6365    0.5403    130070
                  CDU     0.5090    0.4832    0.4957    100514
                  CSU     0.6668    0.2983    0.4122     21044
            Die Linke     0.5818    0.5193    0.5488     98403
                  FDP     0.7248    0.4310    0.5406     54510
                  SPD     0.4796    0.4851    0.4823    106470

             accuracy                         0.5216    541829
            macro avg     0.5720    0.4944    0.5156    541829
         weighted avg     0.5384    0.5216    0.5200    541829



In [40]:
print("********SVM confusion matrix (full tweets, all data)*******")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

********SVM confusion matrix (full tweets, all data)*******


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,18719,3252,2048,65,2209,610,3915
true:Bündnis 90/Die Grünen,2685,82791,12174,908,12682,2556,16274
true:CDU,2738,23890,48564,873,7414,1656,15379
true:CSU,489,5704,3534,6277,1339,1037,2664
true:Die Linke,2941,23016,8730,327,51101,1076,11212
true:FDP,1827,12074,6003,414,4095,23494,6603
true:SPD,3295,25642,14359,549,8993,1987,51645


# DecisionTree

In [41]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     #("chi", SelectKBest(chi2, k=10000)),
                     ("clf", DecisionTreeClassifier(random_state = 111))])

In [42]:
model = pipeline.fit(X_train, y_train)

MemoryError: 

In [ ]:
vectorizer = model.named_steps["vect"]
#chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [ ]:
print(clf.feature_importances_[])

In [ ]:
prediction = model.predict(X_test)

In [ ]:
print("******DecisionTree classification report (full tweets, all data)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

In [ ]:
print("********DecisionTree confusion matrix (full tweets, all data)*******")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

# RandomForest

In [117]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", RandomForestClassifier(random_state = 111))])

In [118]:
model = pipeline.fit(X_train, y_train)

In [119]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [120]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices = True)]
feature_names = np.asarray(feature_names)

In [121]:
prediction = model.predict(X_test)

In [122]:
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

0.5324056346290156
                       precision    recall  f1-score   support

                  AfD     0.4551    0.3262    0.3800      4409
Bündnis 90/Die Grünen     0.4844    0.3987    0.4374     15176
                  CDU     0.5574    0.4867    0.5197     22299
                  CSU     0.5602    0.3600    0.4383      5159
            Die Linke     0.6367    0.6492    0.6429     39751
                  FDP     0.4757    0.3066    0.3729      7596
                  SPD     0.4376    0.6000    0.5061     28351

             accuracy                         0.5324    122741
            macro avg     0.5153    0.4468    0.4710    122741
         weighted avg     0.5378    0.5324    0.5288    122741



In [ ]:
cm = confusion_matrix(y_test, prediction)
print(cm)

# GradientBoostingClassifier

In [45]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     #("chi", SelectKBest(chi2, k=10000)),
                     ("clf", GradientBoostingClassifier(random_state = 111))])

In [46]:
model = pipeline.fit(X_train, y_train)

MemoryError: 

In [ ]:
vectorizer = model.named_steps["vect"]
#chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [ ]:
prediction = model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

In [ ]:
cm = confusion_matrix(y_test, prediction)
print(cm)

# KNeighborsClassifier

In [162]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", GradientBoostingClassifier(random_state = 111))])

In [163]:
model = pipeline.fit(X_train, y_train)

In [164]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [165]:
prediction = model.predict(X_test)

In [166]:
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

0.4868218443714814
                       precision    recall  f1-score   support

                  AfD     0.7035    0.3035    0.4240      4409
Bündnis 90/Die Grünen     0.8557    0.2044    0.3300     15176
                  CDU     0.7738    0.2974    0.4297     22299
                  CSU     0.7589    0.3125    0.4427      5159
            Die Linke     0.4199    0.8940    0.5714     39751
                  FDP     0.7320    0.2075    0.3233      7596
                  SPD     0.5044    0.3512    0.4141     28351

             accuracy                         0.4868    122741
            macro avg     0.6783    0.3672    0.4193    122741
         weighted avg     0.6014    0.4868    0.4534    122741



In [167]:
cm = confusion_matrix(y_test, prediction)
print(cm)

[[ 1338    19   116     7  1996    37   896]
 [   76  3102   355    59 10014   118  1452]
 [  142   134  6632    84 11922    86  3299]
 [    8    16    96  1612  2935    17   475]
 [  163   144   522   142 35536   138  3106]
 [   54    71   117    97  5127  1576   554]
 [  121   139   733   123 17097   181  9957]]


# Multilayer Perceptron Classifier

In [147]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", MLPClassifier(hidden_layer_sizes=(7,7), random_state = 111))])

In [148]:
model = pipeline.fit(X_train, y_train)

In [149]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [150]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices = True)]
feature_names = np.asarray(feature_names)

In [151]:
prediction = model.predict(X_test)

In [152]:
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

0.6169658060468792
                       precision    recall  f1-score   support

                  AfD     0.4159    0.4625    0.4379      4409
Bündnis 90/Die Grünen     0.6867    0.4918    0.5731     15176
                  CDU     0.6433    0.5779    0.6089     22299
                  CSU     0.7732    0.4811    0.5931      5159
            Die Linke     0.6780    0.7435    0.7093     39751
                  FDP     0.6658    0.4406    0.5303      7596
                  SPD     0.5113    0.6333    0.5658     28351

             accuracy                         0.6170    122741
            macro avg     0.6249    0.5472    0.5741    122741
         weighted avg     0.6281    0.6170    0.6154    122741



In [153]:
cm = confusion_matrix(y_test, prediction)
print(cm)

[[ 2039    82   435    37   729   140   947]
 [  219  7464  1129   103  2923   334  3004]
 [  540   721 12887   190  2864   245  4852]
 [  153   206   577  2482   741   136   864]
 [  722   977  1840    85 29554   463  6110]
 [  226   431   501   143  1567  3347  1381]
 [ 1004   989  2663   170  5209   362 17954]]
